# LangServeで遊ぶ

次のコマンドでWebアプリケーションを起動してからノートブックを実行していく。

```sh
poetry run task start
```

In [1]:
import requests
from datetime import datetime

base_url = "http://localhost:8000"

## とりあえずOpenAIでチャット

In [2]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/chat/invoke", json=json)
resp.json()

{'output': {'content': '東京です。',
  'additional_kwargs': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False},
 'callback_events': [],
 'metadata': {'run_id': '79758175-c660-4ff2-9e4f-fd5c061bb93c'}}

In [3]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/chat/stream", json=json, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "9fea404e-75f0-45c6-85fd-095c1b9b379d"}',
 '',
 'event: data',
 'data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"東","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"京","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"です","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"。","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: end',
 '']

## `RunnablePassthrough`で入力と出力を確認してみる

In [4]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

'日本の首都は？'

In [5]:
json = {
    "input": {
        "question": "日本の首都は？"
    }
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

{'question': '日本の首都は？'}

In [6]:
json = {
    "input": [ "日本の首都は？" ]
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

['日本の首都は？']

In [7]:
resp = requests.post(f"{base_url}/passthrough/stream", json={ "input": "日本の首都は？" }, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "a3b0e43d-236c-4407-8599-3556e74bfc5e"}',
 '',
 'event: data',
 'data: "日本の首都は？"',
 '',
 'event: end',
 '']

## `RunnableLambda`

In [8]:
json = {
    "input": {
        "text": "abc",
        "times": 3,
    }
}
resp = requests.post(f"{base_url}/lambda/invoke", json=json)
resp.json()["output"]

'abcabcabc'

In [9]:
json = {
    "input": {
        "text": "abc",
        "times": 3,
    }
}
resp = requests.post(f"{base_url}/lambda/stream", json=json, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "ec18ad94-caeb-4b6e-8283-344dcd76a7ac"}',
 '',
 'event: data',
 'data: "abcabcabc"',
 '',
 'event: end',
 '']

## `RunnableGenerator`でServer-Sent Events

In [10]:
json = {
    "input": {
        "text": "hello",
        "sleep": 1,
    }
}
resp = requests.post(f"{base_url}/generator/stream", json=json, stream=True)
date = datetime.now()
for line in resp.iter_lines(decode_unicode=True):
    td = datetime.now() - date
    print((f"{td.seconds}", line))


('0', 'event: metadata')
('0', 'data: {"run_id": "ac3e22d7-e497-4a0b-ab22-e5b0bd44dd77"}')
('0', '')
('0', 'event: data')
('0', 'data: "h"')
('0', '')
('1', 'event: data')
('1', 'data: "e"')
('1', '')
('2', 'event: data')
('2', 'data: "l"')
('2', '')
('3', 'event: data')
('3', 'data: "l"')
('3', '')
('4', 'event: data')
('4', 'data: "o"')
('4', '')
('5', 'event: end')
('5', '')


In [11]:
json = {
    "input": {
        "text": "hello",
        "sleep": 1,
    }
}
date = datetime.now()
resp = requests.post(f"{base_url}/generator/invoke", json=json, stream=True)
dt = datetime.now() - date
dt, resp.json()

(datetime.timedelta(seconds=5, microseconds=22568),
 {'output': 'hello',
  'callback_events': [],
  'metadata': {'run_id': '247e759e-1597-41ad-b2c9-7c44541cb12b'}})